In [1]:
# Scrape odds from odds portal
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
from pathlib import Path
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox") # linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
PATH = Path('../chromedriver')
driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)
driver.get('https://www.pinnacle.com/en/baseball/mlb/matchups')
time.sleep(3)

tz = pytz.timezone('America/New_York')
today = datetime.now(tz)
yesterday = today - timedelta(days = 1)
year = today.year
month = today.month

# Use yesterdays games to search for results on oddsportal.com
# As OP.com messes with its dates sometimes

In [6]:
# Change timezone to EST
# Change in account settings on pinnacle

games = []

# For loop for each game
# Get the team names
ele = driver.find_element_by_tag_name('div.style_participants__FetcR')
team1, team2 = ele.text.split('\n')

# Get odds
odds = driver.find_elements_by_tag_name('span.price')
odd1, odd2 = float(odds[0].text), float(odds[1].text)

# Save each game as dict
game_dict = {
    'Date': str(today),
    'Home': team1,
    'Vis': team2,
    'Home Odds': odd1,
    'Vis Odds': odd2
}

# Save each game to games array
games.append(game_dict)

# Create dataframe of games and odds
future_games_df = pd.DataFrame(games)

# Save to CSV for Keras
future_games_df.to_csv('auto_files/future_games.csv', index=False)

In [8]:
driver.get('https://www.oddsportal.com/baseball/usa/mlb/results/#/page/1/')

In [9]:
# Change timezone to Eastern USA

time.sleep(3)
timezone = driver.find_elements_by_class_name('user-header-fakeselect')
timezone[1].click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[7]/div/a[40]').click()

In [24]:
odds_list = []

# Scrape yesterdays results and store to csv(same csv as future games)
table = driver.find_element_by_tag_name('tbody')

rows = table.find_elements_by_tag_name('tr')

for row in rows:
    try:
        date = row.find_element_by_css_selector('th.first2.tl').text
    except:
        cols = row.find_elements_by_tag_name('td')

        if len(cols) > 5:
            teams = cols[1].text
            teams = teams.replace('\n', '')
            if ' - ' in teams:
                home_team = teams.split(' - ')[0]
                vis_team = teams.split(' - ')[1]
            score = cols[2].text
            home_odds = cols[3].text
            vis_odds = cols[4].text
            if ':' in score:
                home_score = int(score.split(':')[0])
                vis_score = int(score.split(':')[1])
                outcome = True if home_score > vis_score else False
            
            if 'Yesterday' in date and score:
                odds_dict = {
                    'Date': yesterday,
                    'Home': home_team,
                    'Visitor': vis_team,
                    'Home Odds': home_odds,
                    'Vis Odds': vis_odds,
                    'Home Score': home_score,
                    'Vis Score': vis_score,
                    'Home Win': outcome                   
                }

                odds_list.append(odds_dict)

In [27]:
# Create dataframe of past games and results
past_games_df = pd.DataFrame(odds_list)

# Save file
past_games_df.to_csv('auto_files/yesterdays_results.csv', index=False)